<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/my_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtokentome

     |████████████████████████████████| 1.7MB 4.5MB/s 


In [41]:
import math
import numpy as np
import pandas as pd
import json
import random
import json
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchtext
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
import youtokentome as yttm
from torchtext.data import BucketIterator

In [3]:
device = torch.device('cuda:0')

In [4]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [5]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
!unzip '/content/drive/My Drive/unsupervised.csv.zip' 

Archive:  /content/drive/My Drive/unsupervised.csv.zip
  inflating: unsupervised.csv        


In [7]:
!unzip '/content/drive/My Drive/qa_data.jsonl.zip'

Archive:  /content/drive/My Drive/qa_data.jsonl.zip
  inflating: qa_data.jsonl           
  inflating: __MACOSX/._qa_data.jsonl  


In [8]:
qa_data = list()

with open('qa_data.jsonl') as file_object:
    for line in file_object:
        qa_data.append(json.loads(line.strip()))
file_object.close()
qa_data = qa_data[:math.ceil(len(qa_data)*0.4)]#!!!!

In [9]:
train_border = math.ceil(len(qa_data)*0.6)
test_border = math.ceil(len(qa_data)*0.8)

In [10]:
train_df = pd.DataFrame(qa_data[:train_border])
test_df = pd.DataFrame(qa_data[train_border:test_border])
validation_df = pd.DataFrame(qa_data[test_border: ])

In [11]:
train_df

,question,category,responses
0,долго ли идут деньги с яндексденег на карту visa?,"Бизнес, Финансы",[нет. прорыв 35 ;)]
1,можно ли зарегистрировать авто в другом регионе,"Авто, Мото",[можно на родственника из того региона.. . а ...
2,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,"[витамины и умная эмаль (каждый день), ванночк..."
3,в чем отличие медитации от йоги?,Спорт,"[букв в йоге меньше, в медитации ты просто сид..."
4,когда начнут линейку фильмов про лигу чемпионов?,Спорт,[а не фильм? жалко... а я то думал - хорошая к...
...,...,...,...
674110,могут ли в 15 лет взять работать санитаркой в ...,Красота и Здоровье,"[могут на неполный рабочий день., да . с работ..."
674111,можно сократить беременость симки?,"Семья, Дом, Дети","[никак. там она и так короткая. 3дня, ахахахах..."
674112,в каком случае нужно обязательно менять работу?,"Работа, Карьера","[когда деваться некуда..., кода босс сильно да..."
674113,вы часто сами себя накручиваете на ровном мест...,"Знакомства, Любовь, Отношения","[если меня нет рядом, значит мыссленно и душой..."


In [12]:
validation_df["str_responses"] = [" ".join(i) for i in validation_df["responses"]]
train_df["str_responses"] = [" ".join(i) for i in train_df["responses"]]
test_df["str_responses"] = [" ".join(i) for i in test_df["responses"]]

In [13]:
train_df.to_csv(r'train.csv', index = False)
test_df.to_csv(r'test.csv', index = False)
validation_df.to_csv(r'validation.csv', index = False)


In [14]:
# my_train, my_test = train_test_split(my_df, test_size=0.2)

In [15]:
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
    for que in train_df['question']:
        f.write(que + '\n')
f.close()

vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'

In [16]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 7.62 s, sys: 1.02 s, total: 8.65 s
Wall time: 5.4 s


In [17]:
tokenizer = yttm.BPE(model=model_path)

In [18]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [19]:
# tokenized = []

# batch_size = 256

# for i_batch in tqdm(range(math.ceil(len(train_df['question']) / batch_size))):
    
#     tokenized.extend(tokenizer.encode(
#         list(train_df['question'][i_batch*batch_size:(i_batch+1)*batch_size]), bos=True, eos=True))
#     tokenized.extend(tokenizer.encode(
#         list(train_df['str_responses'][i_batch*batch_size:(i_batch+1)*batch_size]), bos=True, eos=True))
    # tokenized.extend(tokenizer.encode(
    # list(my_df['str_responses'][i_batch*batch_size:(i_batch+1)*batch_size]), bos=True, eos=False))

100%|██████████| 2634/2634 [00:23<00:00, 110.34it/s]


In [20]:
from torchtext import data, datasets

In [21]:
def tokenize_question(text):
  return tokenizer.encode(text, bos=True, eos=True, output_type=yttm.OutputType.SUBWORD)[::-1]

def tokenize_answer(text):
  return tokenizer.encode(text, bos=True, eos=True, output_type=yttm.OutputType.SUBWORD)#[::-1]

In [22]:
SRC = torchtext.data.Field(
    tokenize = tokenize_question,
    init_token = '<bos>',
    eos_token = '<eos>',
    lower= True,
    include_lengths = True
)

TRG = torchtext.data.Field(
    tokenize = tokenize_answer,
    init_token = '<bos>',
    eos_token = '<eos>',
    lower= True,
    include_lengths = True
)

fields = {
    'question': ('src', SRC),
    'str_responses': ('trg', TRG)
}

In [23]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path = '.',
    train = 'train.csv',
    test = 'test.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = fields
)

In [32]:
SRC.build_vocab([train_data.src, valid_data.src, test_data.src], max_size=25000, min_freq=3 )


In [33]:
SRC.vocab.freqs.most_common(5)

[('<eos>', 1123522),
 ('<bos>', 1123522),
 ('▁в', 304922),
 ('▁на', 203104),
 ('▁как', 194327)]

In [34]:
TRG.build_vocab([train_data.src, valid_data.src, test_data.src],  min_freq=5)
print(len(TRG.vocab))
len(SRC.vocab)

29835


25004

In [37]:
BATCH_SIZE = 512
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
              (train_data, valid_data, test_data),
              batch_size = BATCH_SIZE,
              sort_within_batch = True,
              sort_key = lambda x : len(x.src),
              device = device,
          )

In [59]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()
    self.input_dim = input_dim
    self.emb_dim = emb_dim
    self.enc_hid_dim = enc_hid_dim
    self.dropout = dropout

    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
    self.fc = nn.Linear(enc_hid_dim*2, dec_hid_dim)
    self.dropout = nn.Dropout(dropout)


  def forward(self, src, src_len):
    embedded = self.dropout(self.embedding(src))

    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)

    packed_outputs, hidden = self.rnn(packed_embedded)

    outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)
    hidden = torch.tanh(self.fc(torch.cat(hidden[-2,:,:], hidden[-1.:,:], dim = 1)))
    return outputs, hidden
    

In [52]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()
    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
    self.v = nn.Parameter(torch.rand(dec_hid_dim))

  def forward(self, hidden, encoder_outputs, mask):

    bach_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    energy = energy.permute(0, 2, 1)
    v = self.v.repeat(batch_size, 1).unsqueeze(1)

    attention = torch.bmm(v, energy).squeeze(1)
    attention = attention.masked_fill(mask == 0, -1e10)

    return F.softmax(attention, dim = 1)



In [61]:
class Decoder(nn.Module):

  def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
    super().__init__()
    
    self.emb_dim = emb_dim
    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.dropout = dropout
    self.attention = attention

    self.embedding = nn.Embedding(output_dim, emb_dim)

    self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
    self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, encoder_outputs, mask):

    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))
    a = self.attention(hidden, encoder_outputs, mask)

    a = a.unsqueeze(1)
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    weighted = torch.bmm(a, encoder_outputs)
    weighted = weighted.per,ute(1, 0, 2)
    rnn_input = torch.cat((embedded, weigthed), dim = 2)
    output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

    assert (output == hidden).all()


    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)
    output = self.out(torch.cat((output, weighted, embedded), dim = 1))

    
    return output, hiden.squeeze(0), a.squeeze91


In [47]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, pad_idx, sos_idx, eos_idx, device):
    super().__init__()
    
    self.encoder = encoder
    self.decoder = decoder
    self.pad_idx = pad_idx
    self.sos_idx = sos_idx
    self.eos_idx = eos_idx
    self.device = device

  def create_mask(self, src):
    mask = (src != self.pad_idx).permute(1, 0)
    return mask
  
  def forward(self, src, src_len, trg, teacher_forcing_ratio=0.5):
    if trg is None:
      assert teacher_forcing_ratio == 0
      inference = Truetrg = torch.zeros((100, src.shape[1])).long().fill_(self.sos_idx).to(src.device)
    else:
      inference = False
    
    batch_size = src.shape[1]
    max_len = trg,shape[0]
    trg_vocab_size = delf.decoder.output_dim

    outputs = torch.zeros(max_len, batch-size, trg_vocab_size).to(self.device)

    attentions = torch.zeros(max_len, batch_size, src.shape[0]).to(self.device)

    encoder_outputs, hidden = self.encoder(src, src_len)

    output = trg[0, :]
    mask = self.create_mask(src)

    for t in range(1, max_len):
      output, hidden, attention = self.decoder(output, hidden, encoder_outputs, mask)
      outputs[t] = outputattentions[t] = attentionteacher_force = random.random() < teacher_forcing_ratio
      top1 = output.max(1)[1]
      output = (trg[t] if teacher_force else top1)
      if inference and output.item() == self.eos_idx:
        return outputs[:t], attentions[:t]




In [55]:
len(SRC.vocab)

25004

In [ ]:
attn = Attention(100, 100)
inp_dim = len(SRC.vocab)
out_dim = len(TRG.vocab)

enc = Encoder(inp_dim, 256, 100, 100, 0.8)
dec = Decoder(out_dim, 256, 100, 100, 0.8, attention=attn)

model = Seq2Seq(enc, dec, SRC.vocab.stoi['<pad>'], TRG.vocab.stoi['<sos>'], TRG.vocab.stoi['<eos>'], device).to(device)


In [24]:
class LanguageModelData(torch.utils.data.Dataset):
    
    def __init__(self, data, max_len, pad_index, eos_index, bos_index):
        
        self.data = data
        
        self.max_len = max_len
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
        
        # self.pad_index = pad_index
        # self.bos_index = bos_index
        # self.eos_index = eos_index
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        sequence = self.data[::2][:self.max_len]
        
        # исходная последовательность
        x = sequence['question'][:self.max_len]
        # нужно предсказать смещенную последовательность
        y = sequence['str_responses'][:self.max_len]

        
        pads_x = [self.pad_index] * (self.max_len - len(x))
        pads_y = [self.pad_index] * (self.max_len - len(y))
        
        x = torch.tensor(x + pads_x).long()
        y = torch.tensor(y + pads_y).long()
        
        return x, y

In [24]:
lengths = np.array([len(x) for x in tokenized])
np.percentile(lengths, q=95)

69.0

In [25]:
tokenizer.vocab()[:15]

['<PAD>',
 '<UNK>',
 '<BOS>',
 '<EOS>',
 '▁',
 'о',
 'а',
 'е',
 'т',
 'и',
 'н',
 'с',
 'к',
 'р',
 'в']

In [26]:
batch_size = 64

max_len = 32
bos_index = 2
pad_index = 0
eos_index = 3

In [27]:
random.shuffle(tokenized)

validation_start_index = int(len(tokenized) * 0.05)

In [28]:
train_dataset = LanguageModelData(data=tokenized[:-validation_start_index], max_len=max_len, 
                                  pad_index=pad_index, eos_index=eos_index, bos_index=bos_index)
validation_dataset = LanguageModelData(data=tokenized[-validation_start_index:], max_len=max_len,
                                       pad_index=pad_index, eos_index=eos_index, bos_index=bos_index)

len(train_dataset), len(validation_dataset)

(3202046, 168528)

In [59]:
tokenizer.encode('как сделать эту адову домаш 15', bos=True, eos=True, output_type=yttm.OutputType.SUBWORD)

['<BOS>',
 '▁как',
 '▁сделать',
 '▁эту',
 '▁а',
 'до',
 'ву',
 '▁домаш',
 '▁15',
 '<EOS>']

In [30]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)

In [31]:

class LanguageModel(torch.nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, model_dim, num_layers, dropout, padding_idx):
        
        super().__init__()
        
        # просто эмбеддинги
        self.embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, 
                                                  embedding_dim=embedding_dim,
                                                  padding_idx=padding_idx)
        
        # просто lstm
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, 
                                  hidden_size=model_dim,
                                  num_layers=num_layers, 
                                  dropout=0.3,
                                  batch_first=True)
        
        # выходная матрица эмбеддингов
        # количество выходных фичей равно размеру словаря
        # то есть это задача мультиклассовой классификации, но только классов очень много
        self.language_model_head = torch.nn.Linear(in_features=model_dim,
                                                   out_features=vocab_size,
                                                   bias=False)
        
    def forward(self, x):
        
        x = self.embedding_layer(x)
        
        x, _ = self.lstm(x)
        
        # к каждому элементу последовательности применяется выходная матрица эмбеддингов, 
        # которая переводит вектор lstm в предсказание конкретного слова
        x = self.language_model_head(x)
        
        return x

In [32]:
embedding_dim = 128
model_dim = 128
num_layers = 2
dropout = 0.35

In [33]:
model = LanguageModel(vocab_size=vocab_size, embedding_dim=embedding_dim,
                      model_dim=model_dim, num_layers=num_layers,
                      dropout=dropout, padding_idx=pad_index)
model.to(device)

LanguageModel(
  (embedding_layer): Embedding(30000, 128, padding_idx=0)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.3)
  (language_model_head): Linear(in_features=128, out_features=30000, bias=False)
)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [35]:

print(f'Количество обучаемых параметров в сети: {count_parameters(model):,}')

Количество обучаемых параметров в сети: 7,944,192


In [ ]:
x = x.to(device)
y = y.to(device)